In [1]:
import pandas as pd
import numpy as np

In [2]:
qpd_df = pd.read_csv("data/exports/qinn_processed_data.csv")
df = qpd_df.copy()

In [3]:
df.sample()

,room_num,room_type,customer_id,stay_duration,total_price,price,tax,customer_name,checkin_datetime,checkout_datetime,...,checkin_weekday,checkin_day_name,checkin_time_period,checkout_month,checkout_day,checkout_year,checkout_time,checkout_weekday,checkout_day_name,checkout_time_period
5363,116,K,TX37159417,N,75.0,64.1,10.9,JAKISHA WELLS,2023-09-28 11:00:00,2023-09-29 11:00:00,...,3,Thursday,Morning,9,29,2023,11:00:00,4,Friday,Morning


In [4]:
df.groupby(['checkin_year','checkin_month'])['total_price'].sum()

checkin_year  checkin_month
2022          10               13402.34
              11               24283.33
              12               31711.62
2023          1                21984.73
              2                17359.28
              3                21070.20
              4                23116.47
              5                20981.48
              6                34604.91
              7                40508.55
              8                36533.06
              9                30241.80
              10                3095.00
Name: total_price, dtype: float64

In [5]:
tp0 = df[df['total_price'] == 0]

In [6]:
# Fill missing values with mean 
df['fill_mean'] = df['total_price']
df['fill_mean'] = df['fill_mean'].replace(0.00, np.nan)

In [7]:
df['fill_p25'] = df.groupby(['checkin_year','checkin_month', 'room_type', 'stay_duration'])['fill_mean'].transform(lambda x: x.fillna(x.quantile(.25)))

df.loc[df.total_price == 0.00, ['total_price', 'room_type', 'stay_duration', 'fill_p25']]

,total_price,room_type,stay_duration,fill_p25
12,0.0,NK,N,60.000
42,0.0,K,N,70.000
44,0.0,NK,N,51.675
54,0.0,K,N,60.000
57,0.0,K,N,67.500
...,...,...,...,...
6465,0.0,K,N,50.000
6549,0.0,NQQ,N,70.000
6559,0.0,NK,N,70.000
6561,0.0,K,N,50.000


In [8]:
adj_df = df.copy()

In [9]:
delta = format(adj_df['fill_p25'].sum() - adj_df['total_price'].sum(), ".2f")
print(f"Total price difference: ${delta}")

Total price difference: $45839.99


In [10]:
# Since we took over.
adj_df[(adj_df.checkin_year == 2023)&(adj_df.checkin_month >= 6)].groupby(['checkin_year','checkin_month'])['fill_p25'].sum()

checkin_year  checkin_month
2023          6                34844.91
              7                41177.40
              8                36698.06
              9                30301.80
              10                3095.00
Name: fill_p25, dtype: float64

In [11]:
# Group by 'checkin_year' and 'checkin_month' and calculate the sum of 'total_price' and 'fill_mean'
monthly_adj_df = adj_df[adj_df.checkin_year == 2023].groupby(['checkin_year','checkin_month']).agg({'total_price': 'sum', 'fill_p25': 'sum'})

# Calculate the difference between the sums of each month for 'total_price' and 'fill_mean'
monthly_adj_df['diff_sum'] = monthly_adj_df['fill_p25'] - monthly_adj_df['total_price']

In [12]:
monthly_adj_df

total_price    fill_p25   diff_sum
checkin_year checkin_month                                    
2023         1                 21984.73  30012.6675  8027.9375
             2                 17359.28  23144.4800  5785.2000
             3                 21070.20  25622.5300  4552.3300
             4                 23116.47  29946.0825  6829.6125
             5                 20981.48  26531.4550  5549.9750
             6                 34604.91  34844.9100   240.0000
             7                 40508.55  41177.4000   668.8500
             8                 36533.06  36698.0600   165.0000
             9                 30241.80  30301.8000    60.0000
             10                 3095.00   3095.0000     0.0000